## In this notebook I will implement the training with the Contrastive-Divergence loss

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys


sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/")
sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/model")
from encoded_protein_dataset_new import get_embedding, EncodedProteinDataset_new, collate_fn_new#, dynamic_collate_fn
from get_samples_potts import diverge, loglik_potts
import torch, torchvision
from potts_decoder import PottsDecoder
from torch.utils.data import DataLoader, RandomSampler
from functools import partial
import biotite.structure
from biotite.structure.io import pdbx, pdb
from biotite.structure.residues import get_residues
from biotite.structure import filter_backbone
from biotite.structure import get_chains
from biotite.sequence import ProteinSequence
from typing import Sequence, Tuple, List
import scipy
from tqdm import tqdm
import pandas as pd
import csv
import time
from torch.utils.tensorboard import SummaryWriter

#import pytorch_warmup as warmup
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR

##TURIN HPC
#sys.path.insert(1, "/Data/silva/esm/")

## EUROPA
#sys.path.insert(1, "/home/lucasilva/esm/")

##Lucas computer
sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/esm/")
import esm
#from esm.inverse_folding import util
import esm.pretrained as pretrained
from ioutils import read_fasta, read_encodings
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict
from Bio import SeqIO

#from dynamic_cluster_nce import dynamic_collate_fn, dynamic_cluster
from dynamic_loader import dynamic_collate_fn, dynamic_cluster



In [2]:
### IDEA: MSAS PROCEDURE CAN GIVE DIFFERENT OUTPUT SHAPES? ASK
max_msas = None
#msa_dir = "/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/msas/"
msa_dir = "/home/luchinoprince/split2/"
encoding_dir ="/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/"


train_dataset = EncodedProteinDataset_new(os.path.join(msa_dir, 'train'), encoding_dir, noise=0.02, max_msas=max_msas)          ## Default value of noise used
sequence_test_dataset = EncodedProteinDataset_new(os.path.join(msa_dir, 'test/sequence'), encoding_dir, noise=0.0, max_msas=max_msas)
structure_test_dataset = EncodedProteinDataset_new(os.path.join(msa_dir, 'test/structure'), encoding_dir, noise=0.0, max_msas=max_msas)
superfamily_test_dataset = EncodedProteinDataset_new(os.path.join(msa_dir, 'test/superfamily'), encoding_dir, noise=0.0, max_msas=max_msas)

print(f"I have loaded the train and test datasets: train:{len(train_dataset)}, seq:{len(sequence_test_dataset)}, struc:{len(structure_test_dataset)}, super:{len(superfamily_test_dataset)}")

/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoded_protein_dataset_new.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxh02.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/1vwxh02_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/3o58E00.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/3o58E00_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/2wc7A01.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/2wc7A01_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxP00.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/1vwxP00_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxG01.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/tr

In [10]:
batch_structure_size = 2   ### I think with empty GPU we can go up to 16 easily
perc_subset_test = 1.0     ## During the training, for every dataset available we select a random 10% of its samples
batch_msa_size = 128 ### old is 32, original is 16
q = 21 ##isn't always 21
#dynamic_collate_fn

collate_fn = partial(dynamic_collate_fn, q=q, batch_size=batch_structure_size, batch_msa_size=batch_msa_size)
train_loader = DataLoader(train_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=True,
num_workers=4, pin_memory=True)


sequence_test_loader = DataLoader(sequence_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
num_workers=4, pin_memory=True, sampler=RandomSampler(sequence_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(sequence_test_dataset))))

structure_test_loader = DataLoader(structure_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
num_workers=4, pin_memory=True, sampler=RandomSampler(structure_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(structure_test_dataset))))

superfamily_test_loader = DataLoader(superfamily_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
num_workers=4, pin_memory=True, sampler=RandomSampler(superfamily_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(superfamily_test_dataset))))

In [4]:
decoder = None
embedding = None
torch.cuda.empty_cache()

seed = 24877
torch.random.manual_seed(seed)
np.random.seed(seed)



update_steps = 21000                                      ##Usual values are update steps=10^5, test_steps=10^2
test_steps = 700
bk_iter = False                                                  ## This tells us how ofter we save a model(default values is every ten-thousand updates)
#n_epochs = update_steps//(len(train_dataset)//batch_structure_size)   ## the other update steps will be used for "partial epochs", I want to save the last complet epoch
#print(f"With update_steps:{update_steps} we will do {n_epochs} full epochs")

input_encoding_dim = 512
param_embed_dim = 512
n_param_heads = 48
d_model = 512 ##old 512
n_heads = 8 ## old 8
n_layers = 6
## Check before running which is the GPU which is free the most and put it as the running device
device = 0        ## DON'T SET TO ONE OTHER THAN IN SPECIAL SPECIAL OCCASIONS, VERY NOISYYYYY!
dropout = 0.1

decoder = PottsDecoder(q, n_layers, d_model, input_encoding_dim, param_embed_dim, n_heads, n_param_heads, dropout=dropout)
decoder.to(device)
embedding = get_embedding(q)
embedding.to(device)

512
512


Embedding(22, 21)

In [5]:
def get_loss_new(decoder, inputs, nsteps=1):
    """nsteps is the number of MC steps we take, default is 1 """
    msas, encodings, padding_mask  = [input.to(device, non_blocking=True) for input in inputs]
    B, M, N = msas.shape
    #print(f"encodings' shape{encodings.shape}, padding mask:{padding_mask.shape}")
    param_embeddings, fields = decoder.forward_new(encodings, padding_mask)
    llik_old = torch.mean(loglik_potts(param_embeddings, fields, embedding(msas)))
    #llik_new = diverge(decoder, param_embeddings, fields, msas, embedding, padding_mask, steps=nsteps)
    msas_new = diverge(decoder, param_embeddings, fields, msas, embedding, padding_mask, steps=nsteps)
    llik_new = torch.mean(loglik_potts(param_embeddings, fields, embedding(msas_new)))
    print(llik_new)
    print(llik_old)
    loss = llik_new - llik_old
    print(loss)
    return loss

def get_loss_loader(decoder, loader, nsteps=1):
    decoder.eval()
    losses = []
    #with torch.no_grad():
    for effective_batch_size, inputs_packed in loader:
        div_full = 0
        for inputs in inputs_packed:
            mini_batch_size = inputs[0].shape[0]
            #_, npll = get_loss_indep(decoder, inputs, eta_J, eta_h) ## For independent model without couplings
            div = get_loss_new(decoder, inputs, nsteps=nsteps).item()
            div_full += div * mini_batch_size/effective_batch_size
        losses.append(div_full)
            #del inputs
    
    return np.mean(losses)

def train(decoder, inputs_packed, optimizer, scaler, nsteps=1):
    effective_batch_size = inputs_packed[0]
    loss_div_full = 0
    optimizer.zero_grad(set_to_none=True)                           ## set previous gradients to 0
    with torch.cuda.amp.autocast():  ## autocasting mixed precision
        for inputs in inputs_packed[1]:
            mini_batch_size = inputs[0].shape[0]
            div = get_loss_new(decoder, inputs, nsteps=nsteps)
            div = div * mini_batch_size/effective_batch_size
            ## Get gradients and accumulate them                      
            scaler.scale(div).backward()
            loss_div_full += div.detach()
    
    
    scaler.step(optimizer)
    scaler.update()

    return loss_div_full

In [13]:
#summary_writer = SummaryWriter(log_dir='runs/indep_model')#log_dir=logdir)
#summary_writer = SummaryWriter(log_dir='./runs_CD/')
layout = {
    "metrics": {
        "loss": ["Multiline", ["loss/train", "loss/sequence", "loss/structure", "loss/superfamily"]],}
}
#summary_writer.add_custom_scalars(layout)
lr = 1e-4
optimizer = torch.optim.AdamW(decoder.parameters(), lr=lr)

nsteps=10

start = time.time()
scaler = torch.cuda.amp.GradScaler()
with tqdm(total = update_steps) as pbar: ##This is used to have the nice loading bar while training
    train_loss = 0
    update_step = 0
    max_gpu=0
    train_batch_losses = []
    epoch = 0.0
    while update_step < update_steps:
        for inputs_packed in train_loader:
            decoder.train()

            div_full = train(decoder, inputs_packed, optimizer, scaler, nsteps=nsteps)
            div_full.detach()
            #optimizer.step()                                ## Do a step of GD
            update_step += 1                                ## Increase update step (the update steps will count also different batches within the same epoch)
            epoch = update_step / len(train_loader)
            
            train_batch_losses.append(div_full.item()) ## Here we append the lossess in the different batches within the same epoch
            
            ## We want to keep track of the test loss not at every batch, too costrly otherwise. Usually set to once every 100.
            if (update_step == 1) or (epoch % 1 == 0):
                #check_aux = train_batch_losses.copy()
                #print(check_aux)
                train_loss = np.mean(train_batch_losses)
                with torch.no_grad():
                    #print("beginning first test")
                    sequence_test_loss = get_loss_loader(decoder, sequence_test_loader, nsteps=nsteps)
                    #print("Completed first test")
                    structure_test_loss = get_loss_loader(decoder, structure_test_loader, nsteps=nsteps)
                    #print("Completed second test")
                    superfamily_test_loss = get_loss_loader(decoder, superfamily_test_loader, nsteps=nsteps)
                    #print("Completed third test")

                #summary_writer.add_scalar('loss/train', train_loss, update_step)
                #summary_writer.add_scalar('loss/sequence', sequence_test_loss, update_step)
                #summary_writer.add_scalar('loss/structure', structure_test_loss, update_step)
                #summary_writer.add_scalar('loss/superfamily', superfamily_test_loss, update_step)
                                ## UNCOMMENT THIS!
                train_batch_losses = []

        
            pbar.set_description(f'update_step: {update_step}, epoch: {epoch:.2f}  train: {train_loss:.2f}, sequence: {sequence_test_loss:.2f}, structure: {structure_test_loss:.2f}, superfamily: {superfamily_test_loss:.2f}')
            pbar.update(1)

print(f"It took {time.time()-start} seconds")
save_metrics = {'loss/train': train_loss, 'loss/sequence': sequence_test_loss, 
'loss/structure': structure_test_loss, 'loss/superfamily': superfamily_test_loss}
#summary_writer.add_hparams(hyperparams, save_metrics)
#summary_writer.close()

  0%|          | 0/21000 [00:00<?, ?it/s]/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoded_protein_dataset_new.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))
/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoded_protein_dataset_new.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))
/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoded_protein_dataset_new.py:82: UserWarning: To copy construct from a

KeyboardInterrupt: 

## Let us create a small code that I can deal with easily

In [23]:
iterator=0
while iterator < 2:
    for inputs_packed in train_loader:
        iterator+=1
        print("Iter: ",iterator)
        print(inputs_packed[1][0][0][0,0:5,0:5])
        print("\n")
    #decoder.train()
    #div_full = train(decoder, inputs_packed, optimizer, scaler, nsteps=nsteps)

/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoded_protein_dataset_new.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


Iter:  1
tensor([[20, 20, 20, 20, 20],
        [20, 20, 20, 19,  2],
        [ 2,  4,  3,  9, 12],
        [ 2,  4,  6, 20,  2],
        [ 2,  4,  0,  9, 15]], dtype=torch.int32)




/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoded_protein_dataset_new.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


Iter:  2
tensor([[20, 20, 20, 20, 20],
        [20, 20, 20, 20, 16],
        [ 2,  4,  0,  9, 11],
        [20, 20, 20, 20, 20],
        [20, 20, 20, 19,  2]], dtype=torch.int32)




In [20]:
iterator

1

In [11]:
inputs_packed_saved = []
it = 0
for inputs_packed in train_loader:
    inputs_packed_saved = inputs_packed
    it+=1
    if it > 0:
        break

/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoded_protein_dataset_new.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


In [12]:
inputs_packed_saved[1][0][0].shape

torch.Size([2, 264, 119])

In [14]:
inputs_packed[1][0][2]

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, 

In [8]:
lr = 1e-4
optimizer = torch.optim.AdamW(decoder.parameters(), lr=lr)

nsteps=2
scaler = torch.cuda.amp.GradScaler()
#for it in 
for it in range(100):
    print("Iteration: ", it)
    div_full = train(decoder, inputs_packed_saved, optimizer, scaler, nsteps=nsteps)
    print("\n")

Iteration:  0
tensor(-51.1993, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-51.6796, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4803, device='cuda:0', grad_fn=<SubBackward0>)


Iteration:  1
tensor(-50.5598, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-51.5112, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.9515, device='cuda:0', grad_fn=<SubBackward0>)


Iteration:  2
tensor(-50.6039, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-51.1504, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5465, device='cuda:0', grad_fn=<SubBackward0>)


Iteration:  3
tensor(-51.1805, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-51.5513, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3707, device='cuda:0', grad_fn=<SubBackward0>)


Iteration:  4
tensor(-40.4442, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-40.7716, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3275, device='cuda:0', grad_fn=<SubBackward0>)


Iteration:  5
tensor(-39.9087, device='cuda:0', gr

In [14]:
torch.rand(10, device=0)

tensor([0.8441, 0.3777, 0.9687, 0.6552, 0.9464, 0.5977, 0.2007, 0.8219, 0.5615,
        0.7690], device='cuda:0')